1) Relação básica com o PIB

Use a identidade do PIB pelo lado da demanda:
𝑃𝐼𝐵=𝐶+𝐼+𝐺+𝑋−𝑀

Reduções de tarifas afetam X (exportações), M (importações) e receita tarifária do governo (parte de G / impostos líquidos). Para transformar variações de comércio em PIB (valor adicionado), use um fator de valor adicionado (VA) — nem todo R$1 exportado vira R$1 de PIB.


---


2) Choque 1 — zerar a tarifa de importação do Brasil sobre bens dos EUA (2,7% → 0%)

Preço doméstico de importados cai aproximadamente
Δ𝑃𝑀/𝑃𝑀≈𝑡1+𝑡

Com 𝑡=0,027: queda de ~2,63%.

Variação de quantidade importada (regra de elasticidade, assumindo repasse total do preço):
%Δ𝑄𝑀≈𝜀𝑀⋅𝑡1+𝑡

onde
𝜀𝑀 é a elasticidade de demanda por importações (valor positivo aqui por convenção).

Variação do valor importado (a preços de fronteira) ≈ Δ𝑀≈𝑀0⋅%Δ𝑄𝑀 (pequena aproximação).

Perda de receita tarifária:
Δ𝑇𝑅≈−𝑡⋅𝑀0 (se quiser refinar, ajuste 𝑀0 para o novo volume; como aproximação de 1ª ordem, use o M0).

Efeito no PIB do choque de importações:
Δ𝑃𝐼𝐵𝑀≈−𝜃𝑀⋅Δ𝑀+Δ𝑇𝑅

onde
𝜃𝑀 é o VA doméstico “substituído” nas indústrias que competem com importados (pense como a parcela de valor adicionado local que se perde quando M cresce). Use algo entre 0,25–0,45 conforme o setor.


---


3) Choque 2 — reduzir a tarifa dos EUA sobre exportações do Brasil (50% → 15%)

O preço ao consumidor nos EUA cai de
𝑃(1+0,50) para 𝑃(1+0,15):

Δ𝑃𝑈𝑆=1,15−1,50=−0,2333 (−23,33%)

Variação da quantidade demandada de exportações brasileiras (elasticidade de demanda no mercado dos EUA):
%Δ𝑄𝑋≈𝜀𝑋⋅23,33%

Variação do valor exportado (a preços de fronteira) ≈
Δ𝑋≈𝑋0⋅%Δ𝑄𝑋

Efeito no PIB do choque de exportações:
Δ𝑃𝐼𝐵𝑋≈𝜃𝑋⋅Δ𝑋

onde
𝜃𝑋 é a parcela de VA doméstico nas exportações (valor adicionado doméstico por real exportado). Para manufaturas brasileiras, algo como 0,35–0,55 é comum; ajuste ao seu mix setorial.


---


4) Impacto total no PIB
Δ𝑃𝐼𝐵≈Δ𝑃𝐼𝐵𝑋+Δ𝑃𝐼𝐵𝑀​


In [11]:
# -*- coding: utf-8 -*-
from dataclasses import dataclass, asdict
from typing import Optional, Iterable, List, Dict

try:
    import pandas as pd
except ImportError:
    pd = None

In [12]:
@dataclass
class Inputs:
    # Níveis atuais de comércio (mesma unidade para ambos, ex.: US$ bilhões)
    X0: float  # Exportações do Brasil para os EUA (valor de fronteira)
    M0: float  # Importações do Brasil vindas dos EUA (valor de fronteira)

    # Tarifas (ad valorem) inicial -> nova
    t_import_initial: float = 0.027  # tarifa média de importação BR sobre EUA (2,7%)
    t_import_new: float = 0.0        # zerar

    t_export_initial: float = 0.50   # tarifa média EUA sobre exportações BR (50%)
    t_export_new: float = 0.15       # reduzir para 15%

    # Elasticidades (positivas, por convenção)
    eps_M: float = 0.75  # elasticidade da demanda por importações no BR
    eps_X: float = 0.75  # elasticidade da demanda dos EUA pelas exportações BR

    # Fatores de transformação em valor adicionado (0–1)
    theta_M: float = 0.30  # VA doméstico "substituído" por 1 unidade de M
    theta_X: float = 0.45  # VA doméstico gerado por 1 unidade de X

In [13]:
@dataclass
class Results:
    dP_import_rel: float   # variação percentual do preço dos importados no BR (relativa ao preço bruto inicial)
    dQ_M_rel: float        # variação percentual de quantidade importada
    M_new: float           # novo valor importado
    TR_initial: float      # receita tarifária inicial
    TR_new: float          # nova receita tarifária
    dTR: float             # variação da receita tarifária

    dP_export_rel_US: float  # variação % do preço ao consumidor nos EUA dos bens BR
    dQ_X_rel: float          # variação % da quantidade exportada
    X_new: float             # novo valor exportado

    dM: float              # var (nível) de importações
    dX: float              # var (nível) de exportações
    dPIB_M: float          # contribuição do choque de importações para o PIB
    dPIB_X: float          # contribuição do choque de exportações para o PIB
    dPIB_total: float      # impacto total no PIB

    details: Dict[str, float]  # extras úteis para auditoria

In [14]:
def estimate_gdp_impact(p: Inputs) -> Results:
    """
    Estima o impacto no PIB (back-of-the-envelope) de:
      - Zerar/reduzir tarifa de importação no Brasil sobre bens dos EUA
      - Reduzir tarifa dos EUA sobre exportações do Brasil

    Fórmulas principais (porcentagens em termos relativos ao preço bruto inicial):
      dP_import_rel = (t_new - t_initial) / (1 + t_initial)
      dQ_M_rel = - eps_M * dP_import_rel
      M_new = M0 * (1 + dQ_M_rel)
      TR_initial = t_initial * M0
      TR_new = t_new * M_new
      dTR = TR_new - TR_initial
      dPIB_M = - theta_M * (M_new - M0) + dTR

      dP_export_rel_US = (t_export_new - t_export_initial) / (1 + t_export_initial)
      dQ_X_rel = - eps_X * dP_export_rel_US
      X_new = X0 * (1 + dQ_X_rel)
      dPIB_X = theta_X * (X_new - X0)

      dPIB_total = dPIB_M + dPIB_X
    """

    # --- Importações (Brasil) ---
    dP_import_rel = (p.t_import_new - p.t_import_initial) / (1.0 + p.t_import_initial)  # < 0 se reduzir tarifa
    dQ_M_rel = - p.eps_M * dP_import_rel
    M_new = p.M0 * (1.0 + dQ_M_rel)

    TR_initial = p.t_import_initial * p.M0
    TR_new = p.t_import_new * M_new
    dTR = TR_new - TR_initial

    dM = M_new - p.M0
    dPIB_M = - p.theta_M * dM + dTR

    # --- Exportações (mercado dos EUA) ---
    dP_export_rel_US = (p.t_export_new - p.t_export_initial) / (1.0 + p.t_export_initial)  # < 0 se reduzir tarifa
    dQ_X_rel = - p.eps_X * dP_export_rel_US
    X_new = p.X0 * (1.0 + dQ_X_rel)

    dX = X_new - p.X0
    dPIB_X = p.theta_X * dX

    dPIB_total = dPIB_M + dPIB_X

    details = {
        "pct_drop_import_price_BR(%)": 100.0 * dP_import_rel,
        "pct_change_import_quantity(%)": 100.0 * dQ_M_rel,
        "pct_drop_consumer_price_US_for_BR_goods(%)": 100.0 * dP_export_rel_US,
        "pct_change_export_quantity(%)": 100.0 * dQ_X_rel,
        "import_tariff_initial(%)": 100.0 * p.t_import_initial,
        "import_tariff_new(%)": 100.0 * p.t_import_new,
        "export_tariff_US_initial(%)": 100.0 * p.t_export_initial,
        "export_tariff_US_new(%)": 100.0 * p.t_export_new,
        "theta_M": p.theta_M,
        "theta_X": p.theta_X,
        "eps_M": p.eps_M,
        "eps_X": p.eps_X,
    }

    return Results(
        dP_import_rel=dP_import_rel,
        dQ_M_rel=dQ_M_rel,
        M_new=M_new,
        TR_initial=TR_initial,
        TR_new=TR_new,
        dTR=dTR,
        dP_export_rel_US=dP_export_rel_US,
        dQ_X_rel=dQ_X_rel,
        X_new=X_new,
        dM=dM,
        dX=dX,
        dPIB_M=dPIB_M,
        dPIB_X=dPIB_X,
        dPIB_total=dPIB_total,
        details=details
    )

In [15]:
def format_results(r: Results) -> str:
    lines = []
    lines.append("=== Resultados (mesmas unidades de X0/M0) ===")
    lines.append(f"ΔM (importações)          : {r.dM:,.4f}")
    lines.append(f"ΔX (exportações)          : {r.dX:,.4f}")
    lines.append(f"Receita tarifária inicial : {r.TR_initial:,.4f}")
    lines.append(f"Receita tarifária nova    : {r.TR_new:,.4f}")
    lines.append(f"Δ Receita tarifária (dTR) : {r.dTR:,.4f}")
    lines.append(f"PIB - choque importações  : {r.dPIB_M:,.4f}")
    lines.append(f"PIB - choque exportações  : {r.dPIB_X:,.4f}")
    lines.append(f"ΔPIB total                : {r.dPIB_total:,.4f}")
    lines.append("")
    lines.append("=== Variações percentuais (% relativo ao preço bruto inicial) ===")
    lines.append(f"Preço importado no BR     : {100*r.dP_import_rel:,.3f}%")
    lines.append(f"Qtd. importada (M)        : {100*r.dQ_M_rel:,.3f}%")
    lines.append(f"Preço p/ consumidor nos EUA (bens BR): {100*r.dP_export_rel_US:,.3f}%")
    lines.append(f"Qtd. exportada (X)        : {100*r.dQ_X_rel:,.3f}%")
    return "\n".join(lines)

In [16]:
def grid_sensitivity(
    X0_values: Iterable[float],
    M0_values: Iterable[float],
    eps_X_values: Iterable[float],
    eps_M_values: Iterable[float],
    theta_X_values: Iterable[float],
    theta_M_values: Iterable[float],
    t_imp_initial: float = 0.027,
    t_imp_new: float = 0.0,
    t_exp_initial: float = 0.50,
    t_exp_new: float = 0.15
):
    """
    Gera uma grade de cenários (se pandas estiver disponível, devolve DataFrame).
    """
    rows: List[Dict] = []
    for X0 in X0_values:
        for M0 in M0_values:
            for eps_X in eps_X_values:
                for eps_M in eps_M_values:
                    for theta_X in theta_X_values:
                        for theta_M in theta_M_values:
                            p = Inputs(
                                X0=X0, M0=M0,
                                t_import_initial=t_imp_initial,
                                t_import_new=t_imp_new,
                                t_export_initial=t_exp_initial,
                                t_export_new=t_exp_new,
                                eps_M=eps_M, eps_X=eps_X,
                                theta_M=theta_M, theta_X=theta_X
                            )
                            r = estimate_gdp_impact(p)
                            base = {**asdict(p)}
                            base.update({
                                "dPIB_total": r.dPIB_total,
                                "dPIB_M": r.dPIB_M,
                                "dPIB_X": r.dPIB_X,
                                "dM": r.dM,
                                "dX": r.dX,
                                "M_new": r.M_new,
                                "X_new": r.X_new,
                                "dTR": r.dTR,
                                "pct_drop_import_price_BR(%)": r.details["pct_drop_import_price_BR(%)"],
                                "pct_change_import_quantity(%)": r.details["pct_change_import_quantity(%)"],
                                "pct_drop_consumer_price_US_for_BR_goods(%)": r.details["pct_drop_consumer_price_US_for_BR_goods(%)"],
                                "pct_change_export_quantity(%)": r.details["pct_change_export_quantity(%)"],
                            })
                            rows.append(base)

    if pd is None:
        return rows  # lista de dicionários
    else:
        return pd.DataFrame(rows)

In [17]:
# -------------------------
# Exemplo de uso direto
# -------------------------
if __name__ == "__main__":
    # Exemplo simples (troque pelos seus números reais)
    # Unidades: bilhões (ou milhões) de US$ conforme você preferir,
    # contanto que X0 e M0 usem a mesma unidade.
    params = Inputs(
        X0=40.368,   # Exportações BR->EUA
        M0=40.652,   # Importações BR<-EUA
        t_import_initial=0.027, t_import_new=0.0,
        t_export_initial=0.50,  t_export_new=0.15,
        eps_M=0.75, eps_X=0.75,
        theta_M=0.30, theta_X=0.45
    )
    res = estimate_gdp_impact(params)
    print(format_results(res))

    # Análise de sensibilidade (opcional, requer pandas para DataFrame)
    if pd is not None:
        df = grid_sensitivity(
            X0_values=[35, 40, 45],
            M0_values=[45, 50, 55],
            eps_X_values=[1.0, 1.5, 2.0],
            eps_M_values=[1.0, 1.5, 2.0],
            theta_X_values=[0.35, 0.45, 0.55],
            theta_M_values=[0.25, 0.30, 0.40],
        )
    
    print(df)


=== Resultados (mesmas unidades de X0/M0) ===
ΔM (importações)          : 0.8016
ΔX (exportações)          : 7.0644
Receita tarifária inicial : 1.0976
Receita tarifária nova    : 0.0000
Δ Receita tarifária (dTR) : -1.0976
PIB - choque importações  : -1.3381
PIB - choque exportações  : 3.1790
ΔPIB total                : 1.8409

=== Variações percentuais (% relativo ao preço bruto inicial) ===
Preço importado no BR     : -2.629%
Qtd. importada (M)        : 1.972%
Preço p/ consumidor nos EUA (bens BR): -23.333%
Qtd. exportada (X)        : 17.500%
     X0  M0  t_import_initial  t_import_new  t_export_initial  t_export_new  \
0    35  45             0.027           0.0               0.5          0.15   
1    35  45             0.027           0.0               0.5          0.15   
2    35  45             0.027           0.0               0.5          0.15   
3    35  45             0.027           0.0               0.5          0.15   
4    35  45             0.027           0.0            

1. Elasticidades de comércio (𝜀)
a) 𝜀𝑀: elasticidade da demanda por importações (Brasil)

Mede quanto as importações do Brasil aumentam quando o preço relativo dos importados cai.

Fórmula:
%Δ𝑄𝑀/%Δ𝑃𝑀

Valores típicos:
Estudos empíricos para o Brasil e América Latina costumam encontrar algo entre 0,8 e 2,0, dependendo do setor e do horizonte de tempo.
Por ex.: Blyde (BID, 2014) reporta elasticidades em torno de 1,2–1,7 para bens manufaturados.
No código, usei 1,5 como valor médio ilustrativo.

b) 𝜀𝑋: elasticidade da demanda dos EUA por exportações brasileiras

Mede quanto os consumidores/empresas dos EUA aumentam a demanda pelos bens brasileiros quando o preço cai (via redução de tarifa).
É uma proxy da elasticidade-preço da demanda externa.
Valores típicos na literatura: 1,0–2,0 para manufaturas em mercados competitivos. Mais baixos (~0,5–1,0) para commodities onde há menos substitutos.
Coloquei 1,5 como suposição central.

👉 De onde pegar de verdade:
Pesquisas empíricas (papers, relatórios de OMC, FMI, BID, Banco Mundial). Ou você pode estimar via séries temporais (regressão de volumes importados/exportados sobre preços relativos e PIB dos parceiros).


---


2. Fatores de valor adicionado (𝜃)
a) 𝜃𝑋: valor adicionado doméstico em exportações

Define: quanto de cada 1 dólar exportado corresponde a VA doméstico (salários, lucros, impostos líquidos), em vez de insumos importados.
Fontes: OECD TiVA (Trade in Value Added) e estatísticas da WIOD (World Input-Output Database).

Para o Brasil, o VA doméstico em exportações totais costuma estar entre 40–60%, variando muito por setor (soja = quase 100%; eletrônicos montados = ~20–30%). Por isso usei 0,45 como média ilustrativa.

b) 𝜃𝑀: valor adicionado doméstico substituído quando aumenta importação

Define: se as importações aumentam 1 dólar, quanto de produção doméstica deixa de ser gerada.
É menor que 1, porque parte dos importados apenas substitui outros importados, ou são bens não produzidos localmente.
Tipicamente usado em modelos de insumo-produto: algo como 0,25–0,40 para países emergentes.
Coloquei 0,30.

👉 De onde pegar de verdade:
As Tabelas de Recursos e Usos (TRU) do IBGE permitem calcular coeficientes de importação e de VA setorial. A partir daí você deriva 𝜃𝑀 e 𝜃𝑋 de forma mais sólida.


---